In [1330]:
import pandas as pd
import numpy as np
import itertools
import plotly.express as px
import plotly.graph_objects as go
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

# Import & clean Adidas sales data

In [854]:
#Import sales data
adidas_sales_df = pd.read_csv('../adidas_sales.csv')

In [855]:
adidas_sales_df.head()

,Retailer ID,Invoice Date,Region_ID,State_ID,Product_ID,Price per Unit,Units Sold,Operating Margin,Sales Method,Total Sales,Operating Profit
0,1185732,2020-01-01,RG1,ST1,PD1,50.0,1200,0.50,In-store,60000.0,30000.0
1,1185732,2020-01-02,RG1,ST1,PD2,50.0,1000,0.30,In-store,50000.0,15000.0
2,1185732,2020-01-03,RG1,ST1,PD3,40.0,1000,0.35,In-store,40000.0,14000.0
3,1185732,2020-01-04,RG1,ST1,PD4,45.0,850,0.35,In-store,38250.0,13387.5
4,1185732,2020-01-05,RG1,ST1,PD5,60.0,900,0.30,In-store,54000.0,16200.0


In [752]:
#Check the data types
adidas_sales_df.dtypes

Retailer ID           int64
Invoice Date         object
Region_ID            object
State_ID             object
Product_ID           object
Price per Unit      float64
Units Sold            int64
Operating Margin    float64
Sales Method         object
Total Sales         float64
Operating Profit    float64
dtype: object

In [753]:
#Change the data type of the date column
adidas_sales_df['Invoice Date'] = pd.to_datetime(adidas_sales_df['Invoice Date'])

#Drop Retailer ID, Region_ID, Price per Unit, Units Sold, Operating Margin columns
adidas_sales_df = adidas_sales_df.drop(columns=['Retailer ID', 'Region_ID', 'Price per Unit', 'Units Sold', 'Operating Margin'])

In [754]:
adidas_sales_df

,Invoice Date,State_ID,Product_ID,Sales Method,Total Sales,Operating Profit
0,2020-01-01,ST1,PD1,In-store,60000.0,30000.00
1,2020-01-02,ST1,PD2,In-store,50000.0,15000.00
2,2020-01-03,ST1,PD3,In-store,40000.0,14000.00
3,2020-01-04,ST1,PD4,In-store,38250.0,13387.50
4,2020-01-05,ST1,PD5,In-store,54000.0,16200.00
...,...,...,...,...,...,...
9641,2021-01-24,ST50,PD3,Outlet,3465.0,935.55
9642,2021-01-24,ST50,PD4,Outlet,1683.0,471.24
9643,2021-01-24,ST50,PD5,Outlet,3200.0,896.00
9644,2021-01-24,ST50,PD6,Outlet,4305.0,1377.60


# Explore Adidas sales data

In [755]:
#Plot Total Sales by Date
adidas_sales_df.hvplot.line(x='Invoice Date', y='Total Sales', xlabel='Date', ylabel='Total Sales', title='Total Sales by Date')

:Curve   [Invoice Date]   (Total Sales)

In [756]:
#Group data by the sum of Total Sales per week
total_sales_weekly = adidas_sales_df.groupby(pd.Grouper(key='Invoice Date', freq='W')).sum()

#Group data by the sum of Total Sales per month
total_sales_monthly = adidas_sales_df.groupby(pd.Grouper(key='Invoice Date', freq='M')).sum()

#Group data by the sum of Total Sales per quarter
total_sales_quarterly = adidas_sales_df.groupby(pd.Grouper(key='Invoice Date', freq='Q')).sum()

In [757]:
#Initialize the figure
fig = go.Figure()

#Add the traces
fig.add_trace(go.Scatter(x=total_sales_weekly.index, y=total_sales_weekly['Total Sales'], name='Weekly Sales'))

fig.add_trace(go.Scatter(x=total_sales_monthly.index, y=total_sales_monthly['Total Sales'], name='Monthly Sales'))

fig.add_trace(go.Scatter(x=total_sales_quarterly.index, y=total_sales_quarterly['Total Sales'], name='Quarterly Sales'))

#Create and add dropdown
fig.update_layout(updatemenus=[
    dict(
        active=0,
        buttons=list([
            dict(label="All", method="update", args=[{"visible": [True, True, True]}, {"title": "All Sales"}]),
            dict(label="Weekly", method="update", args=[{"visible": [True, False, False]}, {"title": "Weekly Sales"}]),
            dict(label="Monthly", method="update", args=[{"visible": [False, True, False]}, {"title": "Monthly Sales"}]),
            dict(label="Quarterly", method="update", args=[{"visible": [False, False, True]}, {"title": "Quarterly Sales"}])
        ])
    )
])

#Update the layout and show the figure
fig.update_layout(title_text='Total Sales by Date', title_x=0.5)

In [758]:
total_sales_quarterly

,Total Sales,Operating Profit
Invoice Date,,
2020-03-31,6927761.0,2559891.11
2020-06-30,6442039.0,2383123.95
2020-09-30,7191704.0,2700034.59
2020-12-31,3675821.0,1372455.43
2021-03-31,18770496.0,7113924.66
2021-06-30,23794248.0,9530544.61
2021-09-30,28057525.0,11395237.28
2021-12-31,25301709.0,10167871.38


# Import & clean demographic data

In [1043]:
#Retrieved from: https://apps.bea.gov/regional/downloadzip.cfm
#Import demographic data
demo_df = pd.read_csv('SQINC1__ALL_AREAS_1948_2022.csv')

In [1044]:
demo_df

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1948:Q1,1948:Q2,...,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,"""00000""",United States,,SQINC1,1.0,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,204641.7,210069.4,...,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,"""00000""",United States,,SQINC1,2.0,...,"Population (midperiod, persons) 1/",Number of persons,(NA),(NA),...,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,"""00000""",United States,,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0
3,"""01000""",Alabama,5,SQINC1,1.0,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,2496.0,2595.6,...,242348.1,232629.8,230152.7,265461.3,244059.8,245116.7,248677.4,250483.3,254302.5,257231.8
4,"""01000""",Alabama,5,SQINC1,2.0,...,"Population (midperiod, persons) 1/",Number of persons,(NA),(NA),...,5024115.0,5027375.0,5031760.0,5033508.0,5036858.0,5043548.0,5050555.0,5055254.0,5060373.0,5067413.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,"""98000""",Far West,8,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,69208.0,69573.0,68675.0,75629.0,72498.0,73198.0,73197.0,73031.0,73756.0,74669.0
180,Note: See the included footnote file.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,SQINC1: State quarterly personal income summar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,"Last updated: December 23, 2022--new statistic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1045]:
#Drop the columns that are not needed
demo_df = demo_df.drop(columns=['GeoFIPS', 'Region', 'TableName', 'LineCode', 'IndustryClassification',])

demo_df.columns

Index(['GeoName', 'Description', 'Unit', '1948:Q1', '1948:Q2', '1948:Q3',
       '1948:Q4', '1949:Q1', '1949:Q2', '1949:Q3',
       ...
       '2020:Q2', '2020:Q3', '2020:Q4', '2021:Q1', '2021:Q2', '2021:Q3',
       '2021:Q4', '2022:Q1', '2022:Q2', '2022:Q3'],
      dtype='object', length=302)

In [1046]:
#Drop column years that are not needed
demo_df = demo_df.drop(demo_df.columns[3:-11], axis=1)

#Drop NaN values
demo_df = demo_df.dropna()

demo_df

,GeoName,Description,Unit,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,United States,"Personal income (millions of dollars, seasonal...",Millions of dollars,19013184.9,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,United States,"Population (midperiod, persons) 1/",Number of persons,331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,United States,Per capita personal income (dollars) 2/,Dollars,57390.0,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0
3,Alabama,"Personal income (millions of dollars, seasonal...",Millions of dollars,223030.6,242348.1,232629.8,230152.7,265461.3,244059.8,245116.7,248677.4,250483.3,254302.5,257231.8
4,Alabama,"Population (midperiod, persons) 1/",Number of persons,5021627.0,5024115.0,5027375.0,5031760.0,5033508.0,5036858.0,5043548.0,5050555.0,5055254.0,5060373.0,5067413.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Rocky Mountain,"Population (midperiod, persons) 1/",Number of persons,12527279.0,12560325.0,12598245.0,12638757.0,12673366.0,12711764.0,12756878.0,12802158.0,12844282.0,12887858.0,12934628.0
176,Rocky Mountain,Per capita personal income (dollars) 2/,Dollars,56914.0,60464.0,58482.0,58654.0,65105.0,61482.0,61865.0,63032.0,63699.0,64483.0,65851.0
177,Far West,"Personal income (millions of dollars, seasonal...",Millions of dollars,3676803.4,3928385.4,3947530.0,3893315.9,4281973.3,4100396.5,4137883.4,4135892.3,4123489.4,4161472.7,4211194.0
178,Far West,"Population (midperiod, persons) 1/",Number of persons,56763245.0,56762231.0,56739149.0,56691552.0,56618257.0,56558476.0,56530151.0,56503693.0,56461835.0,56422276.0,56398026.0


In [1047]:
demo_df['GeoName'].unique()

array(['United States', 'Alabama', 'Alaska *', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii *', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'New England', 'Mideast',
       'Great Lakes', 'Plains', 'Southeast', 'Southwest',
       'Rocky Mountain', 'Far West'], dtype=object)

In [1048]:
#Remove the states from the GeoName column
demo_df = demo_df.loc[(demo_df['GeoName'] != 'Alabama') & (demo_df['GeoName'] != 'Alaska *') & (demo_df['GeoName'] != 'Arizona') & (demo_df['GeoName'] != 'Arkansas') & (demo_df['GeoName'] != 'California') & (demo_df['GeoName'] != 'Colorado') & (demo_df['GeoName'] != 'Connecticut') & (demo_df['GeoName'] != 'Delaware') & (demo_df['GeoName'] != 'District of Columbia') & (demo_df['GeoName'] != 'Florida') & (demo_df['GeoName'] != 'Georgia') & (demo_df['GeoName'] != 'Hawaii *') & (demo_df['GeoName'] != 'Idaho') & (demo_df['GeoName'] != 'Illinois') & (demo_df['GeoName'] != 'Indiana') & (demo_df['GeoName'] != 'Iowa') & (demo_df['GeoName'] != 'Kansas') & (demo_df['GeoName'] != 'Kentucky') & (demo_df['GeoName'] != 'Louisiana') & (demo_df['GeoName'] != 'Maine') & (demo_df['GeoName'] != 'Maryland') & (demo_df['GeoName'] != 'Massachusetts') & (demo_df['GeoName'] != 'Michigan') & (demo_df['GeoName'] != 'Minnesota') & (demo_df['GeoName'] != 'Mississippi') & (demo_df['GeoName'] != 'Missouri') & (demo_df['GeoName'] != 'Montana') & (demo_df['GeoName'] != 'Nebraska') & (demo_df['GeoName'] != 'Nevada') & (demo_df['GeoName'] != 'New Hampshire') & (demo_df['GeoName'] != 'New Jersey') & (demo_df['GeoName'] != 'New Mexico') & (demo_df['GeoName'] != 'New York') & (demo_df['GeoName'] != 'North Carolina') & (demo_df['GeoName'] != 'North Dakota') & (demo_df['GeoName'] != 'Ohio') & (demo_df['GeoName'] != 'Oklahoma') & (demo_df['GeoName'] != 'Oregon') & (demo_df['GeoName'] != 'Pennsylvania') & (demo_df['GeoName'] != 'Rhode Island') & (demo_df['GeoName'] != 'South Carolina') & (demo_df['GeoName'] != 'South Dakota') & (demo_df['GeoName'] != 'Tennessee') & (demo_df['GeoName'] != 'Texas') & (demo_df['GeoName'] != 'Utah') & (demo_df['GeoName'] != 'Vermont') & (demo_df['GeoName'] != 'Virginia') & (demo_df['GeoName'] != 'Washington') & (demo_df['GeoName'] != 'West Virginia') & (demo_df['GeoName'] != 'Wisconsin') & (demo_df['GeoName'] != 'Wyoming')]

#Remove the regions from the GeoName column
demo_df = demo_df.loc[(demo_df['GeoName'] != 'New England') & (demo_df['GeoName'] != 'Mideast') & (demo_df['GeoName'] != 'Great Lakes') & (demo_df['GeoName'] != 'Plains') & (demo_df['GeoName'] != 'Southeast') & (demo_df['GeoName'] != 'Southwest') & (demo_df['GeoName'] != 'Rocky Mountain') & (demo_df['GeoName'] != 'Far West')]

In [1049]:
demo_df

,GeoName,Description,Unit,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,United States,"Personal income (millions of dollars, seasonal...",Millions of dollars,19013184.9,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,United States,"Population (midperiod, persons) 1/",Number of persons,331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,United States,Per capita personal income (dollars) 2/,Dollars,57390.0,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0


In [1050]:
#Remove GeoName and Unit columns
demo_df = demo_df.drop(columns=['GeoName', 'Unit'])

#Clean Description column entries
demo_df['Description'] = demo_df['Description'].str.replace(' 1/', '')
demo_df['Description'] = demo_df['Description'].str.replace(' 2/', '')

demo_df

,Description,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
0,"Personal income (millions of dollars, seasonal...",19013184.9,20459375.8,19997807.5,19778315.9,22090041.2,20907855.1,20998895.9,21158043.8,21317801.6,21575362.1,21856480.2
1,"Population (midperiod, persons)",331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0,332693300.0,332994420.0
2,Per capita personal income (dollars),57390.0,61727.0,60308.0,59621.0,66595.0,63018.0,63240.0,63664.0,64113.0,64851.0,65636.0


# Merge Adidas sales data with demographic data

In [784]:
merged_df = demo_df.set_index('Description')

In [768]:
total_sales_quarterly = total_sales_quarterly.transpose()

In [769]:
total_sales_quarterly['2022:Q1'] = np.nan
total_sales_quarterly['2022:Q2'] = np.nan
total_sales_quarterly['2022:Q3'] = np.nan

In [770]:
#Rename total_sales_quarterly columns
total_sales_quarterly.columns = ['2020:Q1', '2020:Q2', '2020:Q3', '2020:Q4', '2021:Q1', '2021:Q2', '2021:Q3', '2021:Q4', '2022:Q1', '2022:Q2', '2022:Q3']

In [771]:
total_sales_quarterly

,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
Total Sales,6927761.00,6442039.00,7191704.00,3675821.00,18770496.00,23794248.00,28057525.00,25301709.00,NaN,NaN,NaN
Operating Profit,2559891.11,2383123.95,2700034.59,1372455.43,7113924.66,9530544.61,11395237.28,10167871.38,NaN,NaN,NaN


In [786]:
merged_df = pd.concat([merged_df, total_sales_quarterly], axis=0)

In [787]:
merged_df

,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3
"Personal income (millions of dollars, seasonally adjusted)",1.901318e+07,2.045938e+07,1.999781e+07,1.977832e+07,2.209004e+07,2.090786e+07,2.099890e+07,2.115804e+07,21317801.6,21575362.1,21856480.2
"Population (midperiod, persons)",3.312959e+08,3.314482e+08,3.315966e+08,3.317343e+08,3.317063e+08,3.317762e+08,3.320500e+08,3.323368e+08,332502197.0,332693300.0,332994420.0
Per capita personal income (dollars),5.739000e+04,6.172700e+04,6.030800e+04,5.962100e+04,6.659500e+04,6.301800e+04,6.324000e+04,6.366400e+04,64113.0,64851.0,65636.0
Total Sales,6.927761e+06,6.442039e+06,7.191704e+06,3.675821e+06,1.877050e+07,2.379425e+07,2.805752e+07,2.530171e+07,NaN,NaN,NaN
Operating Profit,2.559891e+06,2.383124e+06,2.700035e+06,1.372455e+06,7.113925e+06,9.530545e+06,1.139524e+07,1.016787e+07,NaN,NaN,NaN


# Predictive analysis with multiple linear regression

In [818]:
#Today I learned that train_test_split only works with columns rather than rows
merged_df = merged_df.transpose()

In [873]:
#Define feature set (which includes all three demographic columns)
X = merged_df.iloc[:-3, :-2]

#Define target set (which includes the total sales and operating profit columns)
y = merged_df.iloc[:-3, 3:]

In [846]:
#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=31)

## Random Forest Regressor

In [995]:
#Create a random forest regressor
rfr = RandomForestRegressor(n_estimators=100, random_state=0)

In [996]:
#Fit the model to the training data
rfr.fit(X_train, y_train)

#Make predictions using the testing data
rfr_predictions = rfr.predict(X_test)

In [997]:
rfr_mse = mean_squared_error(y_test, rfr_predictions)
rfr_rmse = rfr_mse**.5

print(f'Random Forest Regressor MSE: {rfr_mse}')
print(f'Random Forest Regressor RMSE: {rfr_rmse}')

Random Forest Regressor MSE: 14875752818851.555
Random Forest Regressor RMSE: 3856909.7498971317


In [998]:
#A RMSE of 3856909 means that the model is off by about $3.85 million on average

#Create a dataframe of the predictions
rfr_df = pd.DataFrame(rfr_predictions, columns=['p_Total_Sales', 'p_Operating_Profit'], index=y_test.index)
rfr_df['a_Total_Sales'], rfr_df['a_Operating_Profit'] = y_test['Total Sales'], y_test['Operating Profit']
rfr_df

,p_Total_Sales,p_Operating_Profit,a_Total_Sales,a_Operating_Profit
2021:Q1,19789758.82,7.939638e+06,18770496.0,7113924.66
2020:Q2,13465296.72,5.291072e+06,6442039.0,2383123.95


In [999]:
#Plot the predictions vs. the actual values
px.line(rfr_df, x=rfr_df.index, y=['p_Total_Sales', 'a_Total_Sales', 'p_Operating_Profit', 'a_Operating_Profit'], title='Total Sales Predictions vs. Actual Values')

In [916]:
#Find optimal parameters using GridSearchCV
grid = {'n_estimators': [50, 100, 200, 300, 400], 
        'max_depth': [None, 5, 10, 15, 20], 
        'min_samples_split': [2, 4, 6, 8, 10], 
        'min_samples_leaf': [1, 2, 3, 4, 5], 
        'max_features': ['auto', 'sqrt', 'log2']}

#Grid search function
CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv=5, n_jobs=4)
CV_rfr.fit(X_train, y_train)

c:\Users\mitch\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning:

One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]



GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'n_estimators': [100, 200, 300, 400, 500]})

In [919]:
CV_rfr.best_params_

{'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [1000]:
#Create a random forest regressor
opt_rfr = RandomForestRegressor(max_depth=None, max_features= 'log2', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 50, random_state=0)

In [1001]:
#Fit the model to the training data
opt_rfr.fit(X_train, y_train)

#Make predictions using the testing data
opt_rfr_predictions = opt_rfr.predict(X_test)

In [1002]:
opt_rfr_mse = mean_squared_error(y_test, opt_rfr_predictions)
opt_rfr_rmse = opt_rfr_mse**.5

print(f'Random Forest Regressor MSE: {opt_rfr_mse}')
print(f'Random Forest Regressor RMSE: {opt_rfr_rmse}')

Random Forest Regressor MSE: 13750440415574.021
Random Forest Regressor RMSE: 3708158.6286961916


In [1003]:
#A RMSE of 3708158 means that the model is off by about $3.7 million on average

#Create a dataframe of the predictions
opt_rfr_df = pd.DataFrame(opt_rfr_predictions, columns=['p_Total_Sales', 'p_Operating_Profit'], index=y_test.index)
opt_rfr_df['a_Total_Sales'], opt_rfr_df['a_Operating_Profit'] = y_test['Total Sales'], y_test['Operating Profit']
opt_rfr_df

,p_Total_Sales,p_Operating_Profit,a_Total_Sales,a_Operating_Profit
2021:Q1,19043236.04,7.620859e+06,18770496.0,7113924.66
2020:Q2,13270180.32,5.219826e+06,6442039.0,2383123.95


In [1004]:
#Plot the predictions vs. the actual values
px.line(opt_rfr_df, x=opt_rfr_df.index, y=['p_Total_Sales', 'a_Total_Sales', 'p_Operating_Profit', 'a_Operating_Profit'], title='Total Sales Predictions vs. Actual Values')

## Split data by month rather than quarter

In [1031]:
monthly_merged_df = demo_df.set_index('Description')

In [1037]:
total_sales_monthly = total_sales_monthly.transpose()
total_sales_monthly

Invoice Date,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,...,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31
Total Sales,2312746.00,2140813.00,2474202.00,3193081.00,2164764.0,1084194.00,2182388.00,2641630.00,2367686.00,1428569.00,...,5220782.00,6498339.00,8576956.00,8718953.00,10368031.00,9651596.00,8037898.00,7110189.00,7855390.00,10336130.0
Operating Profit,883774.16,796848.19,879268.76,1275442.09,800635.7,307046.16,681477.61,1042648.29,975908.69,582342.96,...,2067129.24,2644237.72,3456570.88,3429736.01,4098805.98,3852859.51,3443571.79,2987755.39,3101992.69,4078123.3


In [1055]:
#Add empty columns to match the shape of demo_df
total_sales_monthly['2022-01-31'] = np.nan
total_sales_monthly['2022-02-29'] = np.nan
total_sales_monthly['2022-03-31'] = np.nan

total_sales_monthly['2022-04-30'] = np.nan
total_sales_monthly['2022-05-31'] = np.nan
total_sales_monthly['2022-06-30'] = np.nan

total_sales_monthly['2022-07-31'] = np.nan
total_sales_monthly['2022-08-31'] = np.nan
total_sales_monthly['2022-09-30'] = np.nan

In [1051]:
demo_df_elongated = demo_df.set_index('Description')

In [1052]:
quarter_x3 = [2, 3]

for column in demo_df_elongated.columns:
    for quarter in quarter_x3:
            demo_df_elongated[f'{column}_{quarter}'] = demo_df_elongated[column]

In [1053]:
#Sort columns
demo_df_elongated = demo_df_elongated.reindex(sorted(demo_df_elongated.columns), axis=1)

In [1054]:
demo_df_elongated

,2020:Q1,2020:Q1_2,2020:Q1_3,2020:Q2,2020:Q2_2,2020:Q2_3,2020:Q3,2020:Q3_2,2020:Q3_3,2020:Q4,...,2021:Q4_3,2022:Q1,2022:Q1_2,2022:Q1_3,2022:Q2,2022:Q2_2,2022:Q2_3,2022:Q3,2022:Q3_2,2022:Q3_3
Description,,,,,,,,,,,,,,,,,,,,,
"Personal income (millions of dollars, seasonally adjusted)",19013184.9,19013184.9,19013184.9,20459375.8,20459375.8,20459375.8,19997807.5,19997807.5,19997807.5,19778315.9,...,21158043.8,21317801.6,21317801.6,21317801.6,21575362.1,21575362.1,21575362.1,21856480.2,21856480.2,21856480.2
"Population (midperiod, persons)",331295939.0,331295939.0,331295939.0,331448217.0,331448217.0,331448217.0,331596557.0,331596557.0,331596557.0,331734262.0,...,332336782.0,332502197.0,332502197.0,332502197.0,332693300.0,332693300.0,332693300.0,332994420.0,332994420.0,332994420.0
Per capita personal income (dollars),57390.0,57390.0,57390.0,61727.0,61727.0,61727.0,60308.0,60308.0,60308.0,59621.0,...,63664.0,64113.0,64113.0,64113.0,64851.0,64851.0,64851.0,65636.0,65636.0,65636.0


In [1063]:
#Match column names
total_sales_monthly.columns = demo_df_elongated.columns

In [1064]:
total_sales_monthly

,2020:Q1,2020:Q1_2,2020:Q1_3,2020:Q2,2020:Q2_2,2020:Q2_3,2020:Q3,2020:Q3_2,2020:Q3_3,2020:Q4,...,2021:Q4_3,2022:Q1,2022:Q1_2,2022:Q1_3,2022:Q2,2022:Q2_2,2022:Q2_3,2022:Q3,2022:Q3_2,2022:Q3_3
Total Sales,2312746.00,2140813.00,2474202.00,3193081.00,2164764.0,1084194.00,2182388.00,2641630.00,2367686.00,1428569.00,...,10336130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Operating Profit,883774.16,796848.19,879268.76,1275442.09,800635.7,307046.16,681477.61,1042648.29,975908.69,582342.96,...,4078123.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1065]:
#Merge the demographic data with the total sales data
monthly_merged_df = pd.concat([demo_df_elongated, total_sales_monthly], axis=0)
monthly_merged_df

,2020:Q1,2020:Q1_2,2020:Q1_3,2020:Q2,2020:Q2_2,2020:Q2_3,2020:Q3,2020:Q3_2,2020:Q3_3,2020:Q4,...,2021:Q4_3,2022:Q1,2022:Q1_2,2022:Q1_3,2022:Q2,2022:Q2_2,2022:Q2_3,2022:Q3,2022:Q3_2,2022:Q3_3
"Personal income (millions of dollars, seasonally adjusted)",1.901318e+07,1.901318e+07,1.901318e+07,2.045938e+07,20459375.8,2.045938e+07,1.999781e+07,1.999781e+07,1.999781e+07,1.977832e+07,...,21158043.8,21317801.6,21317801.6,21317801.6,21575362.1,21575362.1,21575362.1,21856480.2,21856480.2,21856480.2
"Population (midperiod, persons)",3.312959e+08,3.312959e+08,3.312959e+08,3.314482e+08,331448217.0,3.314482e+08,3.315966e+08,3.315966e+08,3.315966e+08,3.317343e+08,...,332336782.0,332502197.0,332502197.0,332502197.0,332693300.0,332693300.0,332693300.0,332994420.0,332994420.0,332994420.0
Per capita personal income (dollars),5.739000e+04,5.739000e+04,5.739000e+04,6.172700e+04,61727.0,6.172700e+04,6.030800e+04,6.030800e+04,6.030800e+04,5.962100e+04,...,63664.0,64113.0,64113.0,64113.0,64851.0,64851.0,64851.0,65636.0,65636.0,65636.0
Total Sales,2.312746e+06,2.140813e+06,2.474202e+06,3.193081e+06,2164764.0,1.084194e+06,2.182388e+06,2.641630e+06,2.367686e+06,1.428569e+06,...,10336130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Operating Profit,8.837742e+05,7.968482e+05,8.792688e+05,1.275442e+06,800635.7,3.070462e+05,6.814776e+05,1.042648e+06,9.759087e+05,5.823430e+05,...,4078123.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Random Forest Regressor with month split

In [1091]:
monthly_merged_df = monthly_merged_df.transpose()

In [1211]:
#Define feature set (which includes all three demographic columns)
X = monthly_merged_df.iloc[:-9, :3]

#Define target set (which includes the total sales and operating profit columns)
y = monthly_merged_df.iloc[:-9, 3:]

#Prediction sets
X_pred = monthly_merged_df.iloc[-9:, :3]
y_pred = monthly_merged_df.iloc[-9:, 3:]

In [1127]:
#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=31)

In [1128]:
#Create a random forest regressor
rfr = RandomForestRegressor(n_estimators=100, random_state=0)

In [1129]:
#Fit the model to the training data
rfr.fit(X_train, y_train)

#Make predictions using the testing data
rfr_predictions = rfr.predict(X_test)

In [1130]:
rfr_mse = mean_squared_error(y_test, rfr_predictions)
rfr_rmse = rfr_mse**.5

print(f'Random Forest Regressor MSE: {rfr_mse}')
print(f'Random Forest Regressor RMSE: {rfr_rmse}')

Random Forest Regressor MSE: 824943196061.6461
Random Forest Regressor RMSE: 908263.8361520545


In [1178]:
#A RMSE of 908263 means that the model is off by about $900,000 on average

#Create a dataframe of the predictions
rfr_df = pd.DataFrame(rfr_predictions, columns=['p_Total_Sales', 'p_Operating_Profit'], index=y_test.index)
rfr_df['a_Total_Sales'], rfr_df['a_Operating_Profit'] = y_test['Total Sales'], y_test['Operating Profit']
rfr_df = rfr_df.sort_index()
rfr_df

,p_Total_Sales,p_Operating_Profit,a_Total_Sales,a_Operating_Profit
2020:Q1,2.215932e+06,8.043960e+05,2312746.0,883774.16
2020:Q2_3,3.066253e+06,1.182672e+06,1084194.0,307046.16
2020:Q3_2,2.300622e+06,8.526891e+05,2641630.0,1042648.29
2020:Q4,1.603434e+06,5.879956e+05,1428569.0,582342.96
2021:Q2,8.395391e+06,3.343996e+06,6498339.0,2644237.72
2021:Q4_2,8.708044e+06,3.537770e+06,7855390.0,3101992.69


In [1179]:
#Plot the predictions vs. the actual values
px.line(rfr_df, x=rfr_df.index, y=['p_Total_Sales', 'a_Total_Sales', 'p_Operating_Profit', 'a_Operating_Profit'], title='Total Sales Predictions vs. Actual Values')

In [1180]:
#Find optimal parameters using GridSearchCV
grid = {'n_estimators': [50, 100, 200, 300, 400], 
        'max_depth': [None, 5, 10, 15, 20], 
        'min_samples_split': [2, 4, 6, 8, 10], 
        'min_samples_leaf': [1, 2, 3, 4, 5], 
        'max_features': ['auto', 'sqrt', 'log2']}

#Grid search function
CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv=5, n_jobs=4)
CV_rfr.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'n_estimators': [50, 100, 200, 300, 400]})

In [1181]:
CV_rfr.best_params_

{'max_depth': 5,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 50}

In [1205]:
#Create a random forest regressor
opt_rfr = RandomForestRegressor(max_depth=5, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 4, n_estimators= 50, random_state=0)

In [1218]:
#Fit the model to the training data
opt_rfr.fit(X_train, y_train)

#Make predictions using the testing data
opt_rfr_predictions = opt_rfr.predict(X_test)

In [1207]:
opt_rfr_mse = mean_squared_error(y_test, opt_rfr_predictions)
opt_rfr_rmse = opt_rfr_mse**.5

print(f'Random Forest Regressor MSE: {opt_rfr_mse}')
print(f'Random Forest Regressor RMSE: {opt_rfr_rmse}')

Random Forest Regressor MSE: 592093138632.1812
Random Forest Regressor RMSE: 769475.8856729567


In [1208]:
#A RMSE of 3708158 means that the model is off by about $3.7 million on average

#Create a dataframe of the predictions
opt_rfr_df = pd.DataFrame(opt_rfr_predictions, columns=['p_Total_Sales', 'p_Operating_Profit'], index=y_test.index)
opt_rfr_df['a_Total_Sales'], opt_rfr_df['a_Operating_Profit'] = y_test['Total Sales'], y_test['Operating Profit']
opt_rfr_df = opt_rfr_df.sort_index()
opt_rfr_df

,p_Total_Sales,p_Operating_Profit,a_Total_Sales,a_Operating_Profit
2020:Q1,2.038694e+06,7.402727e+05,2312746.0,883774.16
2020:Q2_3,2.828291e+06,1.092221e+06,1084194.0,307046.16
2020:Q3_2,2.183755e+06,8.078539e+05,2641630.0,1042648.29
2020:Q4,1.778379e+06,6.435760e+05,1428569.0,582342.96
2021:Q2,7.905664e+06,3.160981e+06,6498339.0,2644237.72
2021:Q4_2,8.604739e+06,3.490633e+06,7855390.0,3101992.69


In [1209]:
#Plot the predictions vs. the actual values
px.line(opt_rfr_df, x=opt_rfr_df.index, y=['p_Total_Sales', 'a_Total_Sales', 'p_Operating_Profit', 'a_Operating_Profit'], title='Total Sales Predictions vs. Actual Values')

In [1219]:
#Make predictions using the testing data
opt_rfr_predictions_future = opt_rfr.predict(X_pred)

In [1256]:
#Plot total_sales_monthly and opt_rfr_predictions_future
px.line(monthly_merged_df, x=monthly_merged_df.index, y=['Total Sales', 'Operating Profit'], title='Actual Sales With Predicted Sales')\
    .add_scatter(x=monthly_merged_df.iloc[-9:].index, y=opt_rfr_predictions_future[:, 0], mode='lines', name='Total Sales Predictions')\
    .add_scatter(x=monthly_merged_df.iloc[-9:].index, y=opt_rfr_predictions_future[:, 1], mode='lines', name='Operating Profit Predictions')

## Support Vector Regression

### Support Vector Regression with monthly split

In [1325]:
#Create model
svr_quarterly_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

In [1326]:
#Fit model to training data
svr_quarterly_model.fit(X_train, y_train.drop('Operating Profit', axis=1).values.ravel())

#Make predictions using the testing data
svr_quarterly_model_predictions = svr_quarterly_model.predict(X_test)

In [1327]:
#Create a dataframe of the predictions
svr_quarterly_model_df = pd.DataFrame(svr_quarterly_model_predictions, columns=['p_Total_Sales'], index=y_test.index).sort_index()

#Add the actual values to the dataframe
svr_quarterly_model_df['a_Total_Sales'] = y_test['Total Sales']

In [1328]:
svr_quarterly_model_df

,p_Total_Sales,a_Total_Sales
2020:Q1,5668937.5,2312746.0
2020:Q2_3,5668937.5,1084194.0
2020:Q3_2,5668937.5,2641630.0
2020:Q4,5668937.5,1428569.0
2021:Q2,5669337.5,6498339.0
2021:Q4_2,5669337.5,7855390.0


In [1329]:
#plot the predictions vs. the actual values
px.line(svr_quarterly_model_df, x=svr_quarterly_model_df.index, y=['p_Total_Sales', 'a_Total_Sales'], title='Total Sales Predictions vs. Actual Values')

## Decision Tree Regression

In [1414]:
#Create model
dtr_model = DecisionTreeRegressor(max_depth=5, random_state=0)

In [1415]:
#Fit model to training data
dtr_model.fit(X_train, y_train)

#Make predictions using the testing data
dtr_predictions = dtr_model.predict(X_test)

In [1416]:
#Accuracy of the model
dtr_mse = mean_squared_error(y_test, dtr_predictions)
dtr_rmse = dtr_mse**.5

print(f'Random Forest Regressor MSE: {dtr_mse}')
print(f'Random Forest Regressor RMSE: {dtr_rmse}')

Random Forest Regressor MSE: 798809380945.2006
Random Forest Regressor RMSE: 893761.3668900668


In [1417]:
#Create a dataframe of the predictions
dtr_predictions_df = pd.DataFrame(dtr_predictions, columns=['p_Total_Sales', 'p_Operating_Profit'], index=y_test.index)
dtr_predictions_df['a_Total_Sales'], dtr_predictions_df['a_Operating_Profit'] = y_test['Total Sales'], y_test['Operating Profit']
dtr_predictions_df = dtr_predictions_df.sort_index()
dtr_predictions_df

,p_Total_Sales,p_Operating_Profit,a_Total_Sales,a_Operating_Profit
2020:Q1,2307507.5,838058.475,2312746.0,883774.16
2020:Q2_3,2678922.5,1038038.895,1084194.0,307046.16
2020:Q3_2,2275037.0,828693.150,2641630.0,1042648.29
2020:Q4,1123626.0,395056.235,1428569.0,582342.96
2021:Q2,8647954.5,3443153.445,6498339.0,2644237.72
2021:Q4_2,8723159.5,3532939.345,7855390.0,3101992.69


In [1418]:
#Plot the predictions vs. the actual values
px.line(dtr_predictions_df, x=dtr_predictions_df.index, y=['p_Total_Sales', 'a_Total_Sales', 'p_Operating_Profit', 'a_Operating_Profit'], title='Total Sales Predictions vs. Actual Values')

In [1419]:
#Find optimal parameters using GridSearchCV
dtr_grid = {'max_depth': [None, 5, 10, 15, 20],
            'min_samples_split': [2, 4, 6, 8, 10],
            'min_samples_leaf': [1, 2, 3, 4, 5],
            'max_features': ['auto', 'sqrt', 'log2']}

dtr_CV = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=dtr_grid, cv=5, n_jobs=4)
dtr_CV.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 4, 6, 8, 10]})

In [1420]:
dtr_CV.best_params_

{'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 4}

In [1461]:
#Use the optimal parameters to create a new model
opt_dtr_model = DecisionTreeRegressor(max_depth=5, max_features='auto', min_samples_leaf=4, min_samples_split=10, random_state=0)

In [1462]:
#Fit model to training data
opt_dtr_model.fit(X_train, y_train)

#Make predictions using the testing data
opt_dtr_predictions = opt_dtr_model.predict(X_test)

In [1463]:
#Accuracy of the model
opt_dtr_mse = mean_squared_error(y_test, opt_dtr_predictions)
opt_dtr_rmse = opt_dtr_mse**.5

print(f'Random Forest Regressor MSE: {opt_dtr_mse}')
print(f'Random Forest Regressor RMSE: {opt_dtr_rmse}')

Random Forest Regressor MSE: 382392481711.1699
Random Forest Regressor RMSE: 618378.9143487753


In [1464]:
#Create a dataframe of the predictions
opt_dtr_predictions_df = pd.DataFrame(opt_dtr_predictions, columns=['p_Total_Sales', 'p_Operating_Profit'], index=y_test.index)
opt_dtr_predictions_df['a_Total_Sales'], opt_dtr_predictions_df['a_Operating_Profit'] = y_test['Total Sales'], y_test['Operating Profit']
opt_dtr_predictions_df = opt_dtr_predictions_df.sort_index()
opt_dtr_predictions_df

,p_Total_Sales,p_Operating_Profit,a_Total_Sales,a_Operating_Profit
2020:Q1,2096273.25,7.749617e+05,2312746.0,883774.16
2020:Q2_3,2096273.25,7.749617e+05,1084194.0,307046.16
2020:Q3_2,2096273.25,7.749617e+05,2641630.0,1042648.29
2020:Q4,2096273.25,7.749617e+05,1428569.0,582342.96
2021:Q2,7213281.00,2.800046e+06,6498339.0,2644237.72
2021:Q4_2,9100768.80,3.692223e+06,7855390.0,3101992.69


In [1465]:
#Plot the predictions vs. the actual values
px.line(opt_dtr_predictions_df, x=opt_dtr_predictions_df.index, y=['p_Total_Sales', 'a_Total_Sales', 'p_Operating_Profit', 'a_Operating_Profit'], title='Total Sales Predictions vs. Actual Values')